# Problem 1 
## Predicting Credit Approval 

This Problem has been solved in the following journal article using SOED: 

[Self-Organizing and Error Driven (SOED) artificial neural network for smarter](classifications[https://academic.oup.com/jcde/article/4/4/282/5729001)

We will be using ucimlrepo to import the sample data. Make sure to install it:

`pip install ucimlrepo`


In [1]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
credit_approval = fetch_ucirepo(id=27) 
  
# data (as pandas dataframes) 
X = credit_approval.data.features 
y = credit_approval.data.targets 

print(credit_approval.variables) 


   name     role         type demographic description units missing_values
0   A16   Target  Categorical        None        None  None             no
1   A15  Feature   Continuous        None        None  None             no
2   A14  Feature   Continuous        None        None  None            yes
3   A13  Feature  Categorical        None        None  None             no
4   A12  Feature  Categorical        None        None  None             no
5   A11  Feature   Continuous        None        None  None             no
6   A10  Feature  Categorical        None        None  None             no
7    A9  Feature  Categorical        None        None  None             no
8    A8  Feature   Continuous        None        None  None             no
9    A7  Feature  Categorical        None        None  None            yes
10   A6  Feature  Categorical        None        None  None            yes
11   A5  Feature  Categorical        None        None  None            yes
12   A4  Feature  Categor

In [2]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from soed import SOEDClassifier
from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score

### Filling missing values

In [3]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A15     690 non-null    int64  
 1   A14     677 non-null    float64
 2   A13     690 non-null    object 
 3   A12     690 non-null    object 
 4   A11     690 non-null    int64  
 5   A10     690 non-null    object 
 6   A9      690 non-null    object 
 7   A8      690 non-null    float64
 8   A7      681 non-null    object 
 9   A6      681 non-null    object 
 10  A5      684 non-null    object 
 11  A4      684 non-null    object 
 12  A3      690 non-null    float64
 13  A2      678 non-null    float64
 14  A1      678 non-null    object 
dtypes: float64(4), int64(2), object(9)
memory usage: 81.0+ KB


In [4]:
# Filling Missing Values
variable_df = credit_approval.variables
for c,t in variable_df.set_index('name').drop(index=['A16']).type.to_dict().items():
    if t == 'Categorical':
        X.loc[:,c] = np.where(X[c].isna(),X[c].mode(),X[c])
    if t == 'Continuous':
        X.loc[:,c] = X[c].fillna(X[c].median())


In [5]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A15     690 non-null    int64  
 1   A14     690 non-null    float64
 2   A13     690 non-null    object 
 3   A12     690 non-null    object 
 4   A11     690 non-null    int64  
 5   A10     690 non-null    object 
 6   A9      690 non-null    object 
 7   A8      690 non-null    float64
 8   A7      690 non-null    object 
 9   A6      690 non-null    object 
 10  A5      690 non-null    object 
 11  A4      690 non-null    object 
 12  A3      690 non-null    float64
 13  A2      690 non-null    float64
 14  A1      690 non-null    object 
dtypes: float64(4), int64(2), object(9)
memory usage: 81.0+ KB


### Binary Coding Categorical Attributes

In [6]:
X = pd.get_dummies(X)
dummy_vars = [c for c in X.columns if '_' in c]
X[dummy_vars] = X[dummy_vars].astype(int)

y = np.where(y=='+',1,0)

In [7]:
X

,A15,A14,A11,A8,A3,A2,A13_g,A13_p,A13_s,A12_f,...,A6_w,A6_x,A5_g,A5_gg,A5_p,A4_l,A4_u,A4_y,A1_a,A1_b
0,0,202.0,1,1.25,0.000,30.83,1,0,0,1,...,1,0,1,0,0,0,1,0,0,1
1,560,43.0,6,3.04,4.460,58.67,1,0,0,1,...,0,0,1,0,0,0,1,0,1,0
2,824,280.0,0,1.50,0.500,24.50,1,0,0,1,...,0,0,1,0,0,0,1,0,1,0
3,3,100.0,5,3.75,1.540,27.83,1,0,0,0,...,1,0,1,0,0,0,1,0,0,1
4,0,120.0,0,1.71,5.625,20.17,0,0,1,1,...,1,0,1,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,0,260.0,0,1.25,10.085,21.08,1,0,0,1,...,0,0,0,0,1,0,0,1,0,1
686,394,200.0,2,2.00,0.750,22.67,1,0,0,0,...,0,0,1,0,0,0,1,0,1,0
687,1,200.0,1,2.00,13.500,25.25,1,0,0,0,...,0,0,0,0,1,0,0,1,1,0
688,750,280.0,0,0.04,0.205,17.92,1,0,0,1,...,0,0,1,0,0,0,1,0,0,1


### Standardizing Data

In [8]:
X  = (X - X.mean())/X.std()

In [9]:
X.describe()

,A15,A14,A11,A8,A3,A2,A13_g,A13_p,A13_s,A12_f,...,A6_w,A6_x,A5_g,A5_gg,A5_p,A4_l,A4_u,A4_y,A1_a,A1_b
count,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02,...,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02
mean,1.029772e-17,4.891417e-17,1.029772e-17,1.029772e-16,1.029772e-17,2.059544e-16,-1.699124e-16,5.969460e-17,-2.960595e-17,-1.055516e-16,...,-7.465848e-17,4.633974e-17,-1.055516e-16,-1.544658e-17,-1.029772e-17,-1.544658e-17,-1.055516e-16,-1.029772e-17,-3.604202e-17,3.604202e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.952717e-01,-1.066043e+00,-4.935286e-01,-6.643947e-01,-9.559198e-01,-1.497787e+00,-3.098621e+00,-1.082276e-01,-2.998615e-01,-1.087120e+00,...,-3.195125e-01,-2.412421e-01,-1.782472e+00,-5.387730e-02,-5.557424e-01,-5.387730e-02,-1.782472e+00,-5.557424e-01,-6.609584e-01,-1.510762e+00
25%,-1.952717e-01,-6.014412e-01,-4.935286e-01,-6.150897e-01,-7.550425e-01,-7.456942e-01,3.222565e-01,-1.082276e-01,-2.998615e-01,-1.087120e+00,...,-3.195125e-01,-2.412421e-01,5.602055e-01,-5.387730e-02,-5.557424e-01,-5.387730e-02,5.602055e-01,-5.557424e-01,-6.609584e-01,-1.510762e+00
50%,-1.943120e-01,-1.368388e-01,-4.935286e-01,-3.655762e-01,-4.035072e-01,-2.575087e-01,3.222565e-01,-1.082276e-01,-2.998615e-01,9.185289e-01,...,-3.195125e-01,-2.412421e-01,5.602055e-01,-5.387730e-02,-5.557424e-01,-5.387730e-02,5.602055e-01,-5.557424e-01,-6.609584e-01,6.609584e-01
75%,-1.193615e-01,5.136044e-01,1.233822e-01,1.200038e-01,4.919034e-01,5.221970e-01,3.222565e-01,-1.082276e-01,-2.998615e-01,9.185289e-01,...,-3.195125e-01,-2.412421e-01,5.602055e-01,-5.387730e-02,-5.557424e-01,-5.387730e-02,5.602055e-01,-5.557424e-01,1.510762e+00,6.609584e-01
max,1.899821e+01,1.054901e+01,1.328414e+01,7.851932e+00,4.668645e+00,4.109180e+00,3.222565e-01,9.226400e+00,3.330040e+00,9.185289e-01,...,3.125232e+00,4.139206e+00,5.602055e-01,1.853379e+01,1.796787e+00,1.853379e+01,5.602055e-01,1.796787e+00,1.510762e+00,6.609584e-01


### PCA Transformation

In [10]:
pca = PCA(n_components=23)  # Reduce to 23 dimensions
X_pca = pd.DataFrame(pca.fit_transform(X),columns = [f'PC{i}' for i in range(1,24)])

In [11]:
X_pca.describe()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23
count,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02,...,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02,6.900000e+02
mean,1.647635e-16,-1.235726e-16,-1.029772e-16,-8.238177e-17,4.119088e-17,9.911556e-17,-2.059544e-17,7.208405e-17,1.647635e-16,-6.693519e-17,...,4.119088e-17,1.004028e-16,1.029772e-16,-9.267949e-17,-1.338704e-16,2.008056e-16,4.891417e-17,1.184238e-16,-1.415937e-17,3.346759e-17
std,2.196496e+00,1.862298e+00,1.748996e+00,1.684505e+00,1.504244e+00,1.457849e+00,1.379680e+00,1.343024e+00,1.304737e+00,1.283267e+00,...,1.137893e+00,1.067338e+00,1.048601e+00,1.037334e+00,1.016788e+00,1.007276e+00,9.941120e-01,9.486086e-01,9.289733e-01,9.104386e-01
min,-5.046569e+00,-6.308442e+00,-1.120538e+00,-7.846603e+00,-6.167302e+00,-3.338926e+00,-4.867461e+00,-5.576225e+00,-5.247459e+00,-3.535307e+00,...,-3.139304e+00,-3.359711e+00,-3.191515e+00,-2.783939e+00,-2.785048e+00,-4.337297e+00,-2.880427e+00,-5.648884e+00,-2.666067e+00,-7.059714e+00
25%,-1.152886e+00,-1.339565e+00,-6.039851e-01,-1.193876e+00,-8.666296e-01,-1.200434e+00,-7.433507e-01,-7.162327e-01,-8.883733e-01,-7.192713e-01,...,-7.549724e-01,-8.229073e-01,-6.985335e-01,-3.605853e-01,-6.224672e-01,-5.057239e-01,-6.895676e-01,-5.626868e-01,-5.076593e-01,-5.281457e-01
50%,2.907246e-02,6.186040e-02,-2.500572e-01,-1.836852e-01,2.253946e-01,-3.938136e-02,-1.528551e-01,-9.291286e-02,-2.977320e-02,-6.621645e-02,...,-2.140973e-01,-2.129418e-02,-7.182996e-02,-8.381566e-03,-1.217567e-02,1.393103e-02,-6.350970e-02,-5.657254e-02,-3.084555e-02,-5.557615e-02
75%,1.932911e+00,1.260647e+00,2.160807e-01,8.288204e-01,9.633170e-01,9.405794e-01,4.895679e-01,5.040649e-01,8.234462e-01,5.412146e-01,...,5.806877e-01,6.599459e-01,4.136699e-01,3.884491e-01,4.169731e-01,5.345574e-01,4.835641e-01,5.030928e-01,5.233554e-01,3.885424e-01
max,5.358202e+00,6.930683e+00,3.589772e+01,5.150107e+00,6.662834e+00,6.712885e+00,6.733803e+00,4.452799e+00,3.284663e+00,8.377486e+00,...,4.960387e+00,3.183814e+00,5.769187e+00,2.854004e+00,3.509009e+00,6.549554e+00,2.827021e+00,3.757672e+00,3.475318e+00,5.441682e+00


### Split Data

In [12]:
random_index = np.random.permutation(X.shape[0])

i = int(round(X.shape[0]*0.5))
train_index = random_index[:i]
test_index = random_index[i+1:]

X_train = X_pca.iloc[train_index]
X_test = X_pca.iloc[test_index]

y_train = y[train_index]
y_test = y[test_index]

In [13]:
X_train

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23
292,0.372489,-1.214570,-0.912309,0.242296,0.938013,-1.920745,-1.072137,-0.465045,-0.403039,0.987411,...,-0.867719,-0.555701,-1.323742,-1.382540,-1.001399,-1.152816,-0.720823,-0.280055,-0.254611,0.354639
382,-3.503595,1.395844,0.192574,1.097206,-1.047653,1.876101,0.709010,-1.278794,2.684159,-0.340702,...,-0.807715,-0.796405,-0.292927,-0.320946,0.071795,-0.866350,-0.425419,0.001426,0.679745,-0.166627
675,-0.482176,-1.937800,-0.569027,2.174295,1.280868,-0.100216,0.415945,-0.862893,-0.374581,-1.227117,...,-1.304295,1.126879,0.974441,0.977279,0.368521,-1.131209,-0.184667,-0.257521,-0.512536,1.022416
462,-0.987583,-2.284854,-1.043813,0.282710,0.475923,-1.364839,-0.367732,-1.036895,0.572807,0.811824,...,1.174819,0.502233,0.390562,0.110307,0.061269,-0.386145,0.387329,0.052167,-0.327557,-1.210174
642,-3.803858,1.280012,-0.807279,-1.391239,0.822630,0.381172,-0.640314,-0.568691,-1.312780,-0.870624,...,-0.837347,1.113725,1.041131,0.731415,0.576136,-0.550548,0.413113,-0.698852,-1.567750,1.520770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,0.416097,-0.976586,-0.642439,-0.095145,0.618287,-1.304543,0.053329,-0.483179,0.979302,0.408673,...,-1.362510,-0.514627,3.133851,0.196057,-1.491002,0.028608,0.312589,0.887468,1.039509,-0.608759
321,-4.296458,-3.776537,19.649888,-3.989651,3.409383,1.326722,0.201334,2.435520,-0.297460,-2.031926,...,-2.141001,-3.021408,5.769187,-1.041126,-1.666725,-0.541106,-2.480377,-5.648884,-2.550214,-7.059714
149,3.363455,1.286792,0.568246,1.175394,0.675215,-1.440297,1.108963,0.676816,-0.175754,0.286713,...,1.439471,-0.231379,0.202258,0.038899,0.192207,-0.203486,-0.966106,-0.281952,1.335483,-0.450004
1,3.284897,1.340605,0.590922,1.912987,0.431974,0.549299,0.879971,0.428741,1.582966,-1.022288,...,0.240521,-1.175774,-0.789554,-0.238471,-0.221001,-0.046601,1.455198,0.363300,-0.234700,0.775895


### Model Trainig

In [14]:
soed = SOEDClassifier(mlp_max_iter=10000,som_input_len=X_train.shape[1])
soed.fit(X_train.values,y_train)

Model training complete.


### Model Evaluation

In [15]:
y_proba = soed.predict_proba(X_test.values)
y_pred = soed.predict(X_test.values)

recall = recall_score(y_test,y_pred)
precision = precision_score(y_test,y_pred)
accuracy = accuracy_score(y_test,y_pred)
auc = roc_auc_score(y_test,y_proba[:,1])

performance = {'recall':recall,'precision':precision,'accuracy':accuracy,'auc':auc}

In [16]:
print(performance)

{'recall': 0.8471337579617835, 'precision': 0.7916666666666666, 'accuracy': 0.8284883720930233, 'auc': np.float64(0.8527708709424708)}


### Model Comparison

In [17]:
report_df = pd.DataFrame(index = [f'repeat{i}' for i in range(1,6)],
                         columns = ['DT','MLP','SOED'] )
report_df

,DT,MLP,SOED
repeat1,NaN,NaN,NaN
repeat2,NaN,NaN,NaN
repeat3,NaN,NaN,NaN
repeat4,NaN,NaN,NaN
repeat5,NaN,NaN,NaN


In [18]:
for loop_i in range(1,6):
    print(loop_i)
    random_index = np.random.permutation(X.shape[0])

    i = int(round(X.shape[0]*0.7))
    train_index = random_index[:i]
    test_index = random_index[i+1:]
    
    X_train = X_pca.iloc[train_index]
    X_test = X_pca.iloc[test_index]
    
    y_train = y[train_index]
    y_test = y[test_index]

    #soed
    soed = SOEDClassifier(mlp_max_iter=10000,som_input_len=X_train.shape[1])
    soed.fit(X_train.values,y_train)
    y_proba = soed.predict_proba(X_test.values)
    auc = roc_auc_score(y_test,y_proba[:,1])
    report_df.loc[f'repeat{loop_i}','SOED'] = auc

    #dt
    dt = DecisionTreeClassifier(max_depth=10)
    dt.fit(X_train.values,y_train)
    y_proba = dt.predict_proba(X_test.values)
    auc = roc_auc_score(y_test,y_proba[:,1])
    report_df.loc[f'repeat{loop_i}','DT'] = auc

    #mlp
    mlp = MLPClassifier(max_iter=10000)
    mlp.fit(X_train.values,y_train[:,0])
    y_proba = mlp.predict_proba(X_test.values)
    auc = roc_auc_score(y_test,y_proba[:,1])
    report_df.loc[f'repeat{loop_i}','MLP'] = auc

report_df.loc['Average'] = report_df.mean()

1
Model training complete.
2
Model training complete.
3
Model training complete.
4
Model training complete.
5
Model training complete.


In [19]:
report_df

,DT,MLP,SOED
repeat1,0.74374,0.887038,0.824056
repeat2,0.743391,0.909866,0.803975
repeat3,0.76761,0.874116,0.841631
repeat4,0.785368,0.910078,0.846802
repeat5,0.71928,0.876733,0.829738
Average,0.751878,0.891566,0.829241


We can see that SOED and MLP are performing very similarly. SOED will show its superiority in the decision-making problem. 

# Problem 2
## Deciding Credit Approval 

We must assume some data to use credit approval for a decision-making problem. We will assume that when we make a false negative, we will incur a 1000-dollar mistake cost; when we make a false positive, we will incur a 2-dollar cost of errors.


In [20]:
c0 = np.where(y==0,0,1000)
c1 = np.where(y==1,0,2)
c = np.column_stack((c0,c1))


random_index = np.random.permutation(X.shape[0])

i = int(round(X.shape[0]*0.5))
train_index = random_index[:i]
test_index = random_index[i+1:]

X_train = X_pca.iloc[train_index]
X_test = X_pca.iloc[test_index]

y_train = y[train_index]
y_test = y[test_index]

c_train = c[train_index,:]
c_test = c[test_index,:]

In [21]:
soed = SOEDClassifier(mlp_max_iter=10000,som_input_len=X_train.shape[1])
soed.fit(X_train.values,y_train,c_train)

Model training complete.


In [22]:
y_decide = soed.decide(X_test.values)

In [23]:
df = pd.DataFrame(np.column_stack((y_test,y_decide)),columns = ['Reality','Decision'])
df['FN_cost'] = np.where((df.Decision==0) & (df.Reality==1),1000,0)
df['FP_cost'] = np.where((df.Decision==1) & (df.Reality==0),2,0)
df['cost'] = df.FP_cost + df.FN_cost
df

,Reality,Decision,FN_cost,FP_cost,cost
0,0,1,0,2,2
1,0,1,0,2,2
2,1,1,0,0,0
3,0,1,0,2,2
4,0,0,0,0,0
...,...,...,...,...,...
339,0,0,0,0,0
340,0,0,0,0,0
341,0,0,0,0,0
342,0,1,0,2,2


In [24]:
total_cost = df.cost.sum()
print(f'total cost is {total_cost}.')

total cost is 14176.


In [25]:
df.cost.value_counts()

cost
0       242
2        88
1000     14
Name: count, dtype: int64

### Comparing Models

In [26]:
report_df = pd.DataFrame(index = [f'repeat{i}' for i in range(1,6)],
                         columns = ['DT','MLP','SOED'] )
report_df

,DT,MLP,SOED
repeat1,NaN,NaN,NaN
repeat2,NaN,NaN,NaN
repeat3,NaN,NaN,NaN
repeat4,NaN,NaN,NaN
repeat5,NaN,NaN,NaN


In [27]:
def calc_cost(y_reality,y_decide):
    df = pd.DataFrame(np.column_stack((y_reality,y_decide)),columns = ['Reality','Decision'])
    df['FN_cost'] = np.where((df.Decision==0) & (df.Reality==1),1000,0)
    df['FP_cost'] = np.where((df.Decision==1) & (df.Reality==0),10,0)
    df['cost'] = df.FP_cost + df.FN_cost
    return df.cost.sum()

In [28]:
def get_cost_minimizing_threshold(y_reality,y_prob):
    candidate_df = pd.DataFrame(index=range(99),columns = ['thresh','cost'])
    candidate_df.thresh = np.linspace(0.01,0.99,99)
    candidate_df = candidate_df.set_index('thresh')
    
    for t in candidate_df.index.tolist():
        y_decide = np.where(y_prob>t,1,0)
        candidate_df.loc[t,'cost'] = calc_cost(y_reality,y_decide)
    return candidate_df[candidate_df.cost == candidate_df.cost.min()].index[0]

In [29]:
for loop_i in range(1,6):
    print(loop_i)
    random_index = np.random.permutation(X.shape[0])

    i = int(round(X.shape[0]*0.7))
    train_index = random_index[:i]
    test_index = random_index[i+1:]
    
    X_train = X_pca.iloc[train_index]
    X_test = X_pca.iloc[test_index]
    
    y_train = y[train_index]
    y_test = y[test_index]

    c_train = c[train_index,:]
    c_test = c[test_index,:]

    #soed
    soed = SOEDClassifier(mlp_max_iter=10000,som_input_len=X_train.shape[1])
    soed.fit(X_train.values,y_train,c_train)
    y_decide = soed.decide(X_test.values)
    total_cost= calc_cost(y_test,y_decide)
    report_df.loc[f'repeat{loop_i}','SOED'] = total_cost

    #dt
    dt = DecisionTreeClassifier()
    dt.fit(X_train.values,y_train)

    y_prob = dt.predict_proba(X_train.values)[:,1]
    thresh = get_cost_minimizing_threshold(y_train,y_prob)
    print(f'dt threshod= {thresh}')
    y_prob = dt.predict_proba(X_test.values)[:,1]
    y_decide = np.where(y_prob>thresh,1,0)
    total_cost= calc_cost(y_test,y_decide)
    report_df.loc[f'repeat{loop_i}','DT'] = total_cost

    #mlp
    mlp = MLPClassifier(max_iter=10000)
    mlp.fit(X_train.values,y_train[:,0])
    
    y_prob = mlp.predict_proba(X_train.values)[:,1]
    thresh = get_cost_minimizing_threshold(y_train,y_prob)
    print(f'mlp threshod= {thresh}')
    y_prob = mlp.predict_proba(X_test.values)[:,1]
    y_decide = np.where(y_prob>thresh,1,0)
    total_cost= calc_cost(y_test,y_decide)
    report_df.loc[f'repeat{loop_i}','MLP'] = total_cost


report_df.loc['Average'] = report_df.mean()

1
Model training complete.
dt threshod= 0.01
mlp threshod= 0.39
2
Model training complete.
dt threshod= 0.01
mlp threshod= 0.31
3
Model training complete.
dt threshod= 0.01
mlp threshod= 0.27
4
Model training complete.
dt threshod= 0.01
mlp threshod= 0.17
5
Model training complete.
dt threshod= 0.01
mlp threshod= 0.33


In [30]:
report_df

,DT,MLP,SOED
repeat1,27160,14190,6590
repeat2,23210,14220,9610
repeat3,19210,17160,6510
repeat4,18320,12300,10580
repeat5,26180,13150,11610
Average,22816.0,14204.0,8980.0


We can see that SOED will incur significantly less cost than MLP or DT.